<a href="https://colab.research.google.com/github/royam0820/taskingai/blob/main/taskingai_creation_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TaskingAI - Python SDK - Creation of an Assistant

In [ ]:
!pip install taskingai

In [ ]:
import taskingai
import os


## Environment setup

In [ ]:
from google.colab import userdata
# Set the environment variable to be the value stored in user secrets.
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ["TASKINGAI_API_KEY"] =  userdata.get('TASKINGAI_API_KEY')

NB: in Google Colab, we can use the Secrets store to store APIs so that they are not visible in your code.

Other method, in Visual Studio for exemple, is to create a text file named `.env` that will store you secret api keys like so:

OPENAI_API_KEY=sk-xxxxxxxxxxxx

TASKINGAI_API_KEY=taxxxxxxxxxxx

then you need to install this package:
`pip install python-dotenv` and the in your code load the environment variables `load_dotenv()`

## Taskingai Initialisation

In [ ]:
taskingai.init(api_key=os.environ["TASKINGAI_API_KEY"])

## Creating a collection

In [ ]:
collection = taskingai.retrieval.create_collection(
    name="amrCollection",
    embedding_model_id="TpZAZqW9",    #Gpt4-small-embeddings
    capacity=1000,
)

NB: A collection is part of a RAG system and therefore records that will be added need to be processed via embeddings, hence the reference to an `embedding_model` for the collection we are creating. Here the `embedding_model_id="TpZAZqW9"` refers to a gpt4-small-embeddings,

[API collection creation](https://docs.tasking.ai/api/#tag/Retrieval/manage_collections/create_collection)

## Creating a record for the collection

In [ ]:
from taskingai.retrieval import Record, TokenTextSplitter

In [ ]:
record: Record = taskingai.retrieval.create_record(
    collection_id=collection.collection_id,
    content="Machine learning is a subfield of artificial intelligence...",
    text_splitter=TokenTextSplitter(chunk_size=200, chunk_overlap=20),
    metadata={"file_name": "machine_learning.pdf"},
    type="text"  # specify the appropriate content type
)

NB: In the record creation example above, the `type` can be `"text"`, `"file"` or `"web"`;  the `metadata` is a dictionary. It can be set to empty `{} since we are using a `"text"` record.

The `text_splitter` indicates how to split records into chunks. It cannot change after creation.

[API record creation](https://docs.tasking.ai/api/#tag/Retrieval/manage_collections/create_record)

## Creating an Assistant

In [ ]:
from taskingai.assistant.assistant import AssistantRetrieval, AssistantRetrievalType
from taskingai.assistant.memory import AssistantNaiveMemory

assistant = taskingai.assistant.create_assistant(
    model_id="TpQuiRBb",
    name="amrAssistant",
    retrievals=[
        AssistantRetrieval(
            type=AssistantRetrievalType.COLLECTION,
            id=collection.collection_id,
        )
    ],
    memory=AssistantNaiveMemory(),
)

NB: when creating an assistant you will need the following minimal values:
- a `model_id` here we are using the GPT4 model id.
- a `name`
- a `retrieval`, here we are using a collection defined above based on `"text"`
- `memory`: three types
    - `naive`: uses a straightforward approach to memory management, storing all messages in a chat session.
    - `zero` : is ideal for applications where context isn't required. It treats each interaction as an independent instance.
    - `Message Window` : s optimized for long conversational threads where maintaining recent context is crucial. It avoids token wastage by focusing on the most relevant parts of the conversation.

[Memory documentation](https://docs.tasking.ai/docs/guide/product_modules/assistant/components/memory/)


[API assistant creation](https://docs.tasking.ai/api/#tag/Assistant/manage_collections/delete_assistant)

## Creating a Chat and a message

In [ ]:
# Creating the chat
chat = taskingai.assistant.create_chat(
    assistant_id=assistant.assistant_id,
)

# Creating the message
message = taskingai.assistant.create_message(
    assistant_id=assistant.assistant_id,
    chat_id=chat.chat_id,
    text="In which country is paris?",
)

# Printing only the MessageContent
print(message.content)


text='In which country is paris?'


NB:
[API create chat](https://docs.tasking.ai/api/#tag/Assistant/manage_collections/delete_chat)

## Creating the response

In [ ]:
assistant_message = taskingai.assistant.generate_message(
    assistant_id=assistant.assistant_id,
    chat_id=chat.chat_id,
)

print(f"Assistant: {assistant_message.content.text}")

Assistant: Paris is in France.


NB:
[API generate message](https://docs.tasking.ai/api/#tag/Assistant/manage_collections/generate_message)

## Chat continuation

In [ ]:
taskingai.assistant.create_message(
    assistant_id=assistant.assistant_id,
    chat_id=chat.chat_id,
    text="What to eat there? give me bullets points",
)
assistant_message = taskingai.assistant.generate_message(
    assistant_id=assistant.assistant_id,
    chat_id=chat.chat_id,
)

print(f"Assistant: {assistant_message.content.text}")

Assistant: Absolutely, here are some 'must-try' culinary delights in Paris:

- Croissants and baguettes
- Escargot (snails)
- Coq au vin (chicken in wine sauce)
- Crêpes 
- French pastries like macaroons, éclairs and tarts
- Assortment of French cheeses
- Variety of French wines
- French onion soup
- Ratatouille (vegetable stew)
- Beef Bourguignon (beef stew made with red wine)
- Bouillabaisse (fish stew)
- Foie gras
- Tarte tatin (upside-down caramelized apple tart)
- Quiche Lorraine (a savory pie with bacon, cheese and egg)


# Note
 TaskingAI gives you the ability to switch models quickly in your stateful application. Which is quite nice and handy!

# Ressources
- [Video - TaskingAI Introduction](https://youtu.be/4A5uQoawETU)
- [TaskingAI Documentation](https://docs.tasking.ai/docs/guide/introduction)
- [TaskingAI APIs](https://docs.tasking.ai/api/)
- [TaskingAI Integration](https://docs.tasking.ai/docs/integration/overview)
- [TaskingAI Examples](https://www.tasking.ai/examples)
- [TaskingAI Inference](https://docs.tasking.ai/docs/taskingai-inference/overview)
- [TaskingAI - Github](https://github.com/TaskingAI/)
- [Docker Hosting](https://docs.tasking.ai/docs/guide/self-hosting-with-docker)
- [TaskingAI Forum](https://forum.tasking.ai/)